In [20]:
import mikeio
from mikeio import EUMType, EUMUnit, ItemInfo

import pandas as pd
import numpy as np



**get duration and timestepping of simulation from setup file**

In [2]:
fname = r'D:\OpCIS2019_offline\OpCIS2019_offline_PT_decoupled.mfm'
pfs = mikeio.read_pfs(fname)

c:\source\mikeio\mikeio\pfs.py:194: UserWarning: Keyword type_of_radiation_stress_correction defined multiple times. Value: 1
  warnings.warn(f"Keyword {key} defined multiple times. Value: {val}")


In [12]:
start_time = pfs.FemEngine.TIME.start_time
n_timesteps = pfs.FemEngine.TIME.number_of_time_steps
time_step_interval = pfs.FemEngine.TIME.time_step_interval

# create datetime index
time = pd.date_range(start=start_time, periods=n_timesteps+1, freq=f'{time_step_interval}S')


print(f'simulation starting: {start_time} \nwith a timestep of {time_step_interval} s. \nrunning for {n_timesteps} s until {time[-1]}')

simulation starting: 2020-12-14 00:00:00 
with a timestep of 60.0 s. 
running for 360 s until 2020-12-14 06:00:00


**create dummy particle pulse and write to dfs0**

In [18]:
n_particles = 100
mass_particle = 1e20 # just use high number if number of particles should be determining factor here 

pulse_duration = 10 # in timesteps
pulse_start = 3


# generate particle pulse
pulse = np.zeros(len(time))
pulse[pulse_start:pulse_start+pulse_duration] = n_particles

# put in dataframe for overview
df_pulse=pd.DataFrame(index=time)
df_pulse['Number Particles'] = pulse
df_pulse['Mass Flux'] = df_pulse['Number Particles'] * mass_particle 

df_pulse[:20]

,Number Particles,Mass Flux
2020-12-14 00:00:00,0.0,0.000000e+00
2020-12-14 00:01:00,0.0,0.000000e+00
2020-12-14 00:02:00,0.0,0.000000e+00
2020-12-14 00:03:00,100.0,1.000000e+22
2020-12-14 00:04:00,100.0,1.000000e+22
2020-12-14 00:05:00,100.0,1.000000e+22
2020-12-14 00:06:00,100.0,1.000000e+22
2020-12-14 00:07:00,100.0,1.000000e+22
2020-12-14 00:08:00,100.0,1.000000e+22
2020-12-14 00:09:00,100.0,1.000000e+22


In [38]:
EUMType.Mass_flux.units

[kg per sec,
 gram per day,
 kg per day,
 pound per sec,
 gram per sec,
 mg per sec,
 mu g per sec,
 ton per sec]

In [39]:
# first item: Mass Flux [kg/s] (should be related to mass of individual particle)
item1 = ItemInfo("Mass Flux", EUMType.Mass_flux, EUMUnit.kg_per_sec)
da1 = mikeio.DataArray(df_pulse['Mass Flux'].values, time=time, item=item1)


# second item: number of particles
item2 = ItemInfo("Number Particles", EUMType.Number_of_Particles, EUMUnit.Integer)
da2 = mikeio.DataArray(df_pulse['Number Particles'].values, time=time, item=item2)

ds = mikeio.Dataset([da1, da2])
ds

# write output
ds.to_dfs('particle_pulse.dfs0')